In [0]:
!pip install clarifai==9.9.3
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.9/216.9 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.2/324.2 kB 26.9 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 51.2 MB/s eta 0:00:00
     ━━━━━

In [0]:
!pip install protobuf==4.24.2
!pip show protobuf
dbutils.library.restartPython() 
     

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.4
    Uninstalling protobuf-4.24.4:
      Successfully uninstalled protobuf-4.24.4

[notice] A new release of pip available: 22.2.2 -> 23.3
[notice] To update, run: pip install --upgrade pip
Name: protobuf
Version: 4.24.2
Summary: 
Home-page: https://developers.google.com/protocol-buffers/
Author: protobuf@googlegroups.com
Author-email: protobuf@googlegroups.com
License: 3-Clause BSD License
Location: /local_disk0/.ephemeral_nfs/envs/pythonEnv-3044921f-02e5-4006-903b-fe81fd6a87ff/lib/python3.10/site-packages
Requires: 
Required-by: clarifai-grpc, facets-overview, googleapis-common-protos, grpcio-status


# Writing ClarifaiPySpark SDK functions

### ClarifaiPySpark Dataset class

In [0]:
import json
import uuid
from typing import List
import os
import time
import requests
from clarifai.client.app import App
from clarifai.client.dataset import Dataset
from clarifai.client.input import Inputs
from clarifai.client.user import User
from clarifai.errors import UserError
from clarifai_grpc.grpc.api.resources_pb2 import Text
from google.protobuf.json_format import MessageToJson
from google.protobuf.struct_pb2 import Struct
from pyspark.sql import DataFrame as SparkDataFrame
from pyspark.sql import SparkSession


class Dataset(Dataset):
  """Dataset class provides information about dataset of the app and it inherits from the clarifai SDK Dataset class."""

  def __init__(self, user_id: str = "", app_id: str = "", dataset_id: str = ""):
    """Initializes the Dataset object.

    Args:
        user_id (str): The clarifai user ID of the user.
        app_id (str): Clarifai App ID.
        dataset_id (str): Dataset ID of the dataset inside the clarifai App.

    Example: TODO
    """
    self.user = User(user_id=user_id)
    self.app = App(app_id=app_id)
    #Inputs object - for listannotations
    #input_obj = User(user_id="user_id").app(app_id="app_id").inputs()
    self.user_id = user_id
    self.app_id = app_id
    self.dataset_id = dataset_id
    super().__init__(user_id=user_id, app_id=app_id, dataset_id=dataset_id)

  def upload_dataset_from_csv(self,
                              csv_path: str = "",
                              input_type: str = 'text',
                              csv_type: str = None,
                              labels: bool = True,
                              chunk_size: int = 128) -> None:
    """Uploads dataset into clarifai app from the csv file path.

    Args:
        csv_path (str): CSV file path of the dataset to be uploaded into clarifai App.
        input_type (str): Input type of the dataset whether (Image, text).
        csv_type (str): Type of the csv file contents(url, raw, filepath).
        labels (bool): Give True if labels column present in dataset else False.
        chunk_size (int): chunk size of parallel uploads of inputs and annotations.

    Example: TODO

    Note:
        CSV file supports 'inputid', 'input', 'concepts', 'metadata', 'geopoints' columns.
        All the data in the CSV should be in double quotes.
        metadata should be in single quotes format. Example: "{'key': 'value'}"
        geopoints should be in "long,lat" format.

    """
    ### TODO: Can input column names & extract them to convert to our csv format
    self.upload_from_csv(
        csv_path=csv_path,
        input_type=input_type,
        csv_type=csv_type,
        labels=labels,
        chunk_size=chunk_size)

  def upload_dataset_from_folder(self,
                                 folder_path: str,
                                 input_type: str,
                                 labels: bool = False,
                                 chunk_size: int = 128) -> None:
    """Uploads dataset from folder into clarifai app.

    Args:
        folder_path (str): folder path of the dataset to be uploaded into clarifai App.
        input_type (str): Input type of the dataset whether (Image, text).
        labels (bool): Give True if labels column present in dataset else False.
        chunk_size (int): chunk size of parallel uploads of inputs and annotations.

    Example: TODO

    Note:
        Can provide a volume or S3 path to the folder
        If label is true, then folder name is class name (label)
    """

    self.upload_from_folder(
        folder_path=folder_path, input_type=input_type, labels=labels, chunk_size=chunk_size)

  def get_inputs_from_dataframe(self,
                                dataframe,
                                input_type: str,
                                df_type: str,
                                dataset_id: str = None,
                                labels: str = True) -> List[Text]:
    input_protos = []
    input_obj = Inputs(user_id=self.user_id, app_id=self.app_id)

    for row in dataframe.collect():
      if labels:
        labels_list = row["concepts"].split(',')
        labels = labels_list if len(row['concepts']) > 0 else None
      else:
        labels = None

      if 'metadata' in dataframe.columns:
        if row['metadata'] is not None and len(row['metadata']) > 0:
          metadata_str = row['metadata'].replace("'", '"')
          try:
            metadata_dict = json.loads(metadata_str)
          except json.decoder.JSONDecodeError:
            raise UserError("metadata column in CSV file should be a valid json")
          metadata = Struct()
          metadata.update(metadata_dict)
        else:
          metadata = None
      else:
        metadata = None

      if 'geopoints' in dataframe.columns:
        if row['geopoints'] is not None and len(row['geopoints']) > 0:
          geo_points = row['geopoints'].split(',')
          geo_points = [float(geo_point) for geo_point in geo_points]
          geo_info = geo_points if len(geo_points) == 2 else UserError(
              "geopoints column in CSV file should have longitude,latitude")
        else:
          geo_info = None
      else:
        geo_info = None

      input_id = uuid.uuid4().hex
      text = row["input"] if input_type == 'text' else None
      image = row['input'] if input_type == 'image' else None
      video = row['input'] if input_type == 'video' else None
      audio = row['input'] if input_type == 'audio' else None

      if df_type == 'raw':
        input_protos.append(
            input_obj.get_text_input(
                input_id=input_id,
                raw_text=text,
                dataset_id=dataset_id,
                labels=labels,
                geo_info=geo_info))
      elif df_type == 'url':
        input_protos.append(
            input_obj.get_input_from_url(
                input_id=input_id,
                image_url=image,
                text_url=text,
                audio_url=audio,
                video_url=video,
                dataset_id=dataset_id,
                labels=labels,
                geo_info=geo_info))
      else:
        input_protos.append(
            input_obj.get_input_from_file(
                input_id=input_id,
                image_file=image,
                text_file=text,
                audio_file=audio,
                video_file=video,
                dataset_id=dataset_id,
                labels=labels,
                geo_info=geo_info))

    return input_protos

  def upload_from_dataframe(self,
                            dataframe,
                            input_type: str,
                            df_type: str = None,
                            labels: bool = True,
                            chunk_size: int = 128) -> None:

    if input_type not in ['image', 'text', 'video', 'audio']:
      raise UserError('Invalid input type, it should be image,text,audio or video')

    if df_type not in ['raw', 'url', 'file_path']:
      raise UserError('Invalid csv type, it should be raw, url or file_path')

    if df_type == 'raw' and input_type != 'text':
      raise UserError('Only text input type is supported for raw csv type')

    if not isinstance(dataframe, SparkDataFrame):
      raise UserError('dataframe should be a Spark DataFrame')

    chunk_size = min(128, chunk_size)
    input_obj = input_obj = Inputs(user_id=self.user_id, app_id=self.app_id)
    input_protos = self.get_inputs_from_dataframe(
        dataframe=dataframe,
        df_type=df_type,
        input_type=input_type,
        dataset_id=self.dataset_id,
        labels=labels)
    return (input_obj._bulk_upload(inputs=input_protos, chunk_size=chunk_size))

  def upload_dataset_from_dataloader(self,
                                     task: str,
                                     split: str,
                                     module_dir: str = None,
                                     dataset_loader: str = None,
                                     chunk_size: int = 128) -> None:
    """Uploads dataset using a dataloader function for custom formats.

    Args:
        task (str): task type(text_clf, visual-classification, visual_detection, visual_segmentation, visual-captioning).
        split (str): split type(train, test, val).
        module_dir (str): path to the module directory.
        dataset_loader (str): name of the dataset loader.
        chunk_size (int): chunk size for concurrent upload of inputs and annotations.

    Example: TODO
    """
    self.upload_dataset(task, split, module_dir, dataset_loader, chunk_size)

  def upload_dataset_from_table(self,
                                table_path: str,
                                task: str,
                                split: str,
                                input_type: str,
                                table_type: str,
                                labels: bool,
                                module_dir: str = None,
                                dataset_loader=None,
                                chunk_size: int = 128) -> None:
    """upload dataset into clarifai app from spark tables.

    Args:
        table_path (str): path of the table to be uploaded.
        task (str):
        split (str):
        input_type (str): Input type of the dataset whether (Image, text).
        table_type (str): Type of the table contents (url, raw, filepath).
        labels (bool): Give True if labels column present in dataset else False.
        module_dir (str): path to the module directory.
        dataset_loader (str): name of the dataset loader.
        chunk_size (int): chunk size for concurrent upload of inputs and annotations.
    Note:
        Accepted csv format - input, label
        TODO: dataframe dataloader template
        TODO: Can input column names & extreact them to convert to our csv format
    """

    if dataset_loader:
      self.upload_dataset(task, split, module_dir, dataset_loader, chunk_size)

    else:
      # df = sqlContext.table(table_path)
      csv_path = "./"
      # df.write.option("header", True).option("delimiter",",").csv(csv_path)
      spark = SparkSession.builder.appName('Clarifai-spark').getOrCreate()
      df_delta = spark.read.format("delta").load(table_path)
      df_delta.createTempView("temporary_view")
      # convert the temporary view into a CSV file
      csv_path = table_path.replace(".delta", ".csv")
      spark.sql("SELECT * FROM temporary_view").write.option("header",
                                                             "true").format("csv").save(csv_path)
      self.upload_from_csv(
          csv_path=csv_path,
          input_type=input_type,
          csv_type=table_type,
          labels=labels,
          chunk_size=chunk_size)

  def list_inputs_from_dataset(self, per_page: int = None, input_type: str = None):
    """Lists all the inputs from the app.

    Args:
        dataset_id (str): dataset_id of which the inputs needs to be listed.
        per_page (str): No of response of inputs per page.
        input_type (str): Input type that needs to be displayed (text,image)
        TODO: Do we need input_type ?, since in our case it is image, so probably we can go with default value of "image".

    Examples:
        TODO

    Returns:
        list of inputs.
        """
    input_obj = Inputs(user_id=self.user_id, app_id=self.app_id)
    return list(
        input_obj.list_inputs(
            dataset_id=self.dataset_id, input_type=input_type, per_page=per_page))

  def list_annotations(self, per_page: int = None, input_type: str = None):
    """Lists all the annotations for the inputs in the dataset of a clarifai app.

    Args:
        dataset_id (str): dataset_id of which the inputs needs to be listed.
        per_page (str): No of response of inputs per page.
        input_type (str): Input type that needs to be displayed (text,image)
        TODO: Do we need input_type ?, since in our case it is image, so probably we can go with default value of "image".

    Examples:
        TODO

    Returns:
        list of annotations.
    """
    ### input_ids: list of input_ids for which user wants annotations
    input_obj = Inputs(user_id=self.user_id, app_id=self.app_id)
    all_inputs = list(
        input_obj.list_inputs(
            dataset_id=self.dataset_id, input_type=input_type, per_page=per_page))
    return list(input_obj.list_annotations(batch_input=all_inputs))

  def export_annotations_to_dataframe(self):
    """Export all the annotations from clarifai App into spark dataframe.

    Examples:
        TODO

    Returns:
        spark dataframe with annotations"""

    annotation_list = []
    spark = SparkSession.builder.appName('Clarifai-spark').getOrCreate()
    input_obj = Inputs(user_id=self.user_id, app_id=self.app_id)
    all_inputs = list(input_obj.list_inputs(dataset_id=self.dataset_id))
    response = list(input_obj.list_annotations(batch_input=all_inputs))
    for an in response:
      temp = {}
      temp['annotation'] = json.loads(MessageToJson(an.data))
      if not temp['annotation'] or temp['annotation'] == '{}' or temp['annotation'] == {}:
        continue
      temp['id'] = an.id
      temp['user_id'] = an.user_id
      temp['input_id'] = an.input_id
      created_at = float(f"{an.created_at.seconds}.{an.created_at.nanos}")
      temp['created_at'] = time.strftime('%m/%d/% %H:%M:%5', time.gmtime(created_at))
      modified_at = float(f"{an.modified_at.seconds}.{an.modified_at.nanos}")
      temp['modified_at'] = time.strftime('%m/%d/% %H:%M:%5', time.gmtime(modified_at))
      annotation_list.append(temp)
    df = spark.createDataFrame(annotation_list)
    return df


  def export_images_to_volume(self, path, input_response):
    for resp in input_response:
        imgid = resp.id
        ext = resp.data.image.image_info.format
        url = resp.data.image.url
        img_name = path+'/'+imgid+'.'+ext.lower()
        headers = {
            "Authorization": f"Bearer {os.environ['CLARIFAI_PAT']}"
        }
        response = requests.get(url, headers=headers)
        with open(img_name, "wb") as f:
            f.write(response.content)


  def export_text_to_volume(self, path, input_response):
    for resp in input_response:
        textid = resp.id
        url = resp.data.text.url
        file_name = path+'/'+textid+'.txt'
        enc = resp.data.text.text_info.encoding
        headers = {
            "Authorization": f"Bearer {os.environ['CLARIFAI_PAT']}"
        }
        response = requests.get(url, headers=headers)
        with open(file_name, "a", encoding=enc) as f:
            f.write(response.content.decode())


2023-10-19 15:07:04 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=908675;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=416196;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### ClarifaiPySpark Client class

In [0]:
from clarifai.client.app import App
from clarifai.client.base import BaseClient
from clarifai.client.user import User

# from clarifaipyspark.dataset import Dataset


class ClarifaiPySpark(BaseClient):
  """ClarifaiPySpark inherits the BaseClient class from the clarifai SDK and it initializes the client."""

  def __init__(self, user_id: str = "", app_id: str = ""):
    """Initializes clarifai client object.

    Args:
      - user_id (str): A user ID for authentication.
      - app_id (str): An app ID for the application to interact with.
  """

    self.user = User(user_id=user_id)
    self.app = App(app_id=app_id)
    self.user_id = user_id
    self.app_id = app_id
    super().__init__(user_id=user_id, app_id=app_id)

  def dataset(self, dataset_id):
    """Initializes the dataset method with dataset_id.

    Args:
        dataset_id: The dataset_id within the user app.

    Returns:
           Dataset object for the dataset_id.
      """

    self.dataset_id = dataset_id
    try:
      self.app.dataset(dataset_id=dataset_id)
    except:
      print("Creating a new dataset")
      self.app.create_dataset(dataset_id=dataset_id)

    return Dataset(dataset_id=dataset_id, user_id=self.user_id, app_id=self.app_id)


2023-10-19 15:07:08 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=408457;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=373917;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

# Testing ClarifaiPySpark SDK

### Setting Env variables

In [0]:
import os

os.environ['CLARIFAI_PAT'] = '95978ef1e65e4e1ab8b268e94a49b1e9'

2023-10-19 14:50:40 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=909460;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=703563;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Creating ClarifaiPyspark object & creating/fetching image dataset from app

In [0]:
cspark_obj = ClarifaiPySpark(user_id='mansi_k', app_id='databricks_tester')

dataset_obj = cspark_obj.dataset(dataset_id='dataset1')

2023-10-19 15:18:00 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=459924;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=970693;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Fetching current inputs from image dataset

In [0]:
my_inputs = dataset_obj.list_inputs_from_dataset()

my_inputs

2023-10-19 13:21:49 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=300018;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=361412;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

[id: "c4"
 data {
   image {
     url: "https://data.clarifai.com/orig/users/mansi_k/apps/databricks_tester/inputs/image/55889a69788a8ff9a4aab1e457f7420f"
     hosted {
       prefix: "https://data.clarifai.com"
       suffix: "users/mansi_k/apps/databricks_tester/inputs/image/55889a69788a8ff9a4aab1e457f7420f"
       sizes: "orig"
       sizes: "tiny"
       sizes: "small"
       sizes: "large"
       crossorigin: "use-credentials"
     }
     image_info {
       width: 1600
       height: 1435
       format: "WebP"
       color_mode: "YUV"
     }
   }
   concepts {
     id: "id-cat"
     name: "cat"
     value: 1
     app_id: "databricks_tester"
   }
 }
 created_at {
   seconds: 1697705999
   nanos: 358281000
 }
 modified_at {
   seconds: 1697706002
   nanos: 167299000
 }
 status {
   code: INPUT_DOWNLOAD_SUCCESS
   description: "Download complete"
 },
 id: "c7"
 data {
   image {
     url: "https://data.clarifai.com/orig/users/mansi_k/apps/databricks_tester/inputs/image/042dc758e67c8

### Upload images from volume folder to Clarifai dataset

In [0]:
dataset_obj.upload_dataset_from_folder(folder_path='/Volumes/mansi_test/default/cat', input_type='image', labels=True)

2023-10-19 08:59:58 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=998956;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=287604;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs: 100%|██████████| 1/1 [00:05<00:00,  5.23s/it]


### Export image annotations to spark dataframe

In [0]:
annot_df = dataset_obj.export_annotations_to_dataframe()
annot_df

2023-10-19 13:22:04 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=905988;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=472545;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

DataFrame[annotation: map<string,array<map<string,string>>>, created_at: string, id: string, input_id: string, modified_at: string, user_id: string]

### Write annotations dataframe to volume as delta table

In [0]:
annot_df.write.format("delta").mode("overwrite").save("/Volumes/mansi_test/default/test_vol1/imgAnnsDeltaTable") 


2023-10-19 13:22:10 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=988046;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=853982;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Load annotations delta table from volume

In [0]:
df_delta = spark.read.format("delta").load("/Volumes/mansi_test/default/test_vol1/imgAnnsDeltaTable")
df_delta.show()

2023-10-19 13:22:21 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=82632;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=220713;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+--------------------+-------------------+--------------------+--------+-------------------+-------+
|          annotation|         created_at|                  id|input_id|        modified_at|user_id|
+--------------------+-------------------+--------------------+--------+-------------------+-------+
|{concepts -> [{na...|10/19/% 08:59:   %5|4bf61ec2eae84409a...|      c4|10/19/% 08:59:   %5|mansi_k|
|{concepts -> [{na...|10/19/% 08:59:   %5|5b81affe82764c8cb...|     c10|10/19/% 08:59:   %5|mansi_k|
|{concepts -> [{na...|10/19/% 08:59:   %5|5ddcbbd4583a4d6eb...|      c2|10/19/% 08:59:   %5|mansi_k|
|{concepts -> [{na...|10/19/% 08:59:   %5|7fcf791aabd040ffb...|      c3|10/19/% 08:59:   %5|mansi_k|
|{concepts -> [{na...|10/19/% 08:59:   %5|db097c021b5144aa9...|      c6|10/19/% 08:59:   %5|mansi_k|
|{concepts -> [{na...|10/19/% 08:59:   %5|f9163378eb39453c8...|      c7|10/19/% 08:59:   %5|mansi_k|
|{concepts -> [{na...|10/19/% 08:59:   %5|32853b11e46f401e8...|     c11|10/19/% 08:59:   %5

### Loop through delta table rows & columns

In [0]:
for row in df_delta.collect()[:4]:
    print(row['id'])
    print(row['input_id'])
    print(row['user_id'])
    print(row['created_at'])
    print(row['input_id'])
    print(row['annotation'])
    print("########################\n")

2023-10-19 13:22:31 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=94111;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=605579;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

4bf61ec2eae84409a7e8c4ceac3180ec
c4
mansi_k
10/19/% 08:59:   %5
c4
{'concepts': [{'name': 'cat', 'id': 'id-cat', 'value': '1.0', 'appId': 'databricks_tester'}]}
########################

5b81affe82764c8cb4612111626196a2
c10
mansi_k
10/19/% 08:59:   %5
c10
{'concepts': [{'name': 'cat', 'id': 'id-cat', 'value': '1.0', 'appId': 'databricks_tester'}]}
########################

5ddcbbd4583a4d6ebe63fbf2eb5d0753
c2
mansi_k
10/19/% 08:59:   %5
c2
{'concepts': [{'name': 'cat', 'id': 'id-cat', 'value': '1.0', 'appId': 'databricks_tester'}]}
########################

7fcf791aabd040ffba689710f2c0872a
c3
mansi_k
10/19/% 08:59:   %5
c3
{'concepts': [{'name': 'cat', 'id': 'id-cat', 'value': '1.0', 'appId': 'databricks_tester'}]}
########################



### Testing export_images_to_volume()

In [0]:
dataset_obj.export_images_to_volume(path="/Volumes/mansi_test/default/test_vol1", input_response=my_inputs[:5])

2023-10-19 13:27:28 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=395642;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=377012;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Load test csv file & upload text data into app using dataframe

In [0]:
spark = SparkSession.builder.appName("Clarifai-spark").getOrCreate()
df = spark.read.option("header",True).csv("/Volumes/mansi_test/default/test_vol1/img_data2.csv")
df.show()

2023-10-19 15:18:10 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=617312;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=846478;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+-------+--------------------+--------+
|inputid|               input|concepts|
+-------+--------------------+--------+
|  img11|https://img.freep...|     dog|
|  img21|https://images.un...|    duck|
|  img31|https://mymodernm...|    duck|
+-------+--------------------+--------+



In [0]:
dataset_obj.upload_from_dataframe(dataframe=df, input_type='image', df_type='url', labels=True)

2023-10-19 15:18:20 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=483129;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=60663;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs: 100%|██████████| 1/1 [00:02<00:00,  2.51s/it]


### Creating ClarifaiPyspark object & creating/fetching text dataset from app

In [0]:
cspark_obj = ClarifaiPySpark(user_id='mansi_k', app_id='docQA_tester')

dataset_obj = cspark_obj.dataset(dataset_id='docQA_tester-dataset-1')

2023-10-19 15:07:26 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=445764;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=855606;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Fetching current inputs from text dataset

In [0]:
my_inputs = dataset_obj.list_inputs_from_dataset()

my_inputs

2023-10-19 13:28:06 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=415402;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=260392;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

[id: "t1"
 data {
   concepts {
     id: "id-happy"
     name: "happy"
     value: 1
     app_id: "docQA_tester"
   }
   text {
     url: "https://data.clarifai.com/orig/users/mansi_k/apps/docQA_tester/inputs/text/e58883f24060c49e19d8bc7cf4e5e8f9"
     hosted {
       prefix: "https://data.clarifai.com"
       suffix: "users/mansi_k/apps/docQA_tester/inputs/text/e58883f24060c49e19d8bc7cf4e5e8f9"
       sizes: "orig"
       crossorigin: "use-credentials"
     }
     text_info {
       char_count: 20
       encoding: "UTF8"
     }
   }
 }
 created_at {
   seconds: 1697717839
   nanos: 238509000
 }
 modified_at {
   seconds: 1697717869
   nanos: 119549000
 }
 status {
   code: INPUT_DOWNLOAD_SUCCESS
   description: "Download complete"
 },
 id: "t2"
 data {
   concepts {
     id: "id-neutral"
     name: "neutral"
     value: 1
     app_id: "docQA_tester"
   }
   text {
     url: "https://data.clarifai.com/orig/users/mansi_k/apps/docQA_tester/inputs/text/391ab3ba74427b778e0cd9a2d33d6922"
  

### Testing export_text_to_volume()

Q- Create folders acc to concept name?   
Q- Save text as csv?

In [0]:
dataset_obj.export_text_to_volume(path="/Volumes/mansi_test/default/test_vol1", input_response=my_inputs)

2023-10-19 13:28:17 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=319111;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=633711;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Upload text from volume csv to Clarifai dataset

In [0]:
dataset_obj.upload_dataset_from_csv(csv_path='/Volumes/mansi_test/default/test_vol1/emotions_data.csv', input_type='text', labels=True, csv_type='raw')

2023-10-19 12:17:18 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=856945;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=618624;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs: 100%|██████████| 1/1 [00:41<00:00, 41.02s/it]


### Load test csv file & upload text data into app using dataframe

In [0]:
spark = SparkSession.builder.appName("Clarifai-spark").getOrCreate()
df = spark.read.option("header",True).csv("/Volumes/mansi_test/default/test_vol1/emotions_data1.csv")
df.show()

2023-10-19 15:07:39 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=960477;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=482382;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+-------+--------------------+--------+
|inputid|               input|concepts|
+-------+--------------------+--------+
|    t11|   I am feeling good|   happy|
|    t21|This is a guava tree| neutral|
|    t31|This is a saddeni...|     sad|
+-------+--------------------+--------+



In [0]:
dataset_obj.upload_from_dataframe(dataframe=df, input_type='text', df_type='raw', labels=True)

2023-10-19 15:07:48 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=488728;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=549985;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs: 100%|██████████| 1/1 [00:01<00:00,  1.15s/it]
